In [ ]:
import numpy as np
from PIL import Image
import cv2

pano_img_fname = './R0010003.JPG'
lp_img_fname = './diy_tsi.jpg'

output_size = 5004

In [ ]:
def rabbit_hole_fast(pano_bgr, out_size=1024, yaw_deg=0.0, zoom=1.0, mirror_x=False, background=0):
    
    """
    Rabbit-hole (zenith at center, ground at rim) stereographic projection.
    Vectorized + OpenCV remap for speed. Assumes input is a 2:1 equirectangular panorama.
    """
    H, W = pano_bgr.shape[:2]
    assert abs(W / H - 2.0) < 1e-3, f"Input must be 2:1 equirectangular (got {W}x{H})."

    S = int(out_size)
    c = (S - 1) / 2.0
    R = c

    # Build target grid (image coords), then to normalized plane coords with +Y up
    yy, xx = np.indices((S, S), dtype=np.float32)
    X = (xx - c) / R
    Y = (c - yy) / R

    r = np.sqrt(X*X + Y*Y)
    mask = r <= 1.0

    # Stereographic inverse: theta = 2*atan(r/zoom); lat = +pi/2 - theta
    # zoom>1 reduces theta → more sky in view (shrinks ground ring)
    theta = 2.0 * np.arctan((r / np.maximum(zoom, 1e-8)).astype(np.float32))
    lat = (np.pi/2.0) - theta

    # Azimuth (bearing) + yaw spin
    lon = np.arctan2(Y, X) + np.deg2rad(yaw_deg).astype(np.float32)
    if mirror_x:
        lon = -lon

    # Map to equirectangular pixel coords (float32 for cv2.remap)
    u = (lon / (2*np.pi) + 0.5) * W
    v = (0.5 - lat / np.pi) * H

    # Horizontal wrap: modulo W so remap doesn’t need wrap border
    u = np.mod(u, W).astype(np.float32)
    # Vertical clamp (no wrap vertically)
    v = np.clip(v, 0, H - 1).astype(np.float32)

    # For outside-circle pixels, force to -1 so borderValue is used
    u_out = u.copy(); v_out = v.copy()
    u_out[~mask] = -1e9
    v_out[~mask] = -1e9

    # Remap (bilinear). BORDER_CONSTANT fills outside with background.
    out = cv2.remap(
        pano_bgr, u_out, v_out,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(background, background, background)
    )

    return out


In [ ]:
pano_bgr = cv2.imread(pano_img_fname)
rh = rabbit_hole_fast(pano_bgr, 
                      out_size=output_size, 
                      yaw_deg=20, 
                      zoom=1.1, 
                      mirror_x=False, 
                      background=0)

cv2.imwrite(lp_img_fname, rh)